In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Input, Add, BatchNormalization, GlobalAvgPool2D, ReLU, SeparableConv2D
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.preprocessing import image
from tensorflow.keras.utils import plot_model

In [2]:
def conv_bn(x, filters, kernel_size, strides=1):
    x = Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    return x

In [3]:
def sep_bn(x, filters, kernel_size, strides=1):
    x = SeparableConv2D(filters, kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    return x

In [4]:
def entry_flow(x):
    x = conv_bn(x, filters=32, kernel_size=3, strides=2)
    x = ReLU()(x)

    x = conv_bn(x, filters=64, kernel_size=3)
    x = ReLU()(x)
    
    residual = x
    x = sep_bn(x, filters=128, kernel_size=3)
    x = ReLU()(x)
    x = sep_bn(x, filters=128, kernel_size=3)
    x = MaxPool2D(pool_size=(3, 3), strides=2, padding="same")(x)
    residual = conv_bn(residual, filters=128, kernel_size=1, strides=2)
    x = Add()([residual, x])
    x = ReLU()(x)
    
    residual = x
    x = sep_bn(x, filters=256, kernel_size=3)
    x = ReLU()(x)    
    x = sep_bn(x, filters=256, kernel_size=3)
    x = MaxPool2D(pool_size=(3, 3), strides=2, padding="same")(x) 
    residual = conv_bn(residual, filters=256, kernel_size=1, strides=2)
    x = Add()([residual, x])
    x = ReLU()(x)
    
    residual = x
    x = sep_bn(x, filters=728, kernel_size=3)
    x = ReLU()(x)    
    x = sep_bn(x, filters=728, kernel_size=3)
    x = MaxPool2D(pool_size=(3, 3), strides=2, padding="same")(x) 
    residual = conv_bn(residual, filters=728, kernel_size=1, strides=2)
    x = Add()([residual, x])
    
    return x

In [5]:
def middle_flow(tensor):
    for i in range(8):

        x = ReLU()(tensor)
        x = sep_bn(x, filters=728, kernel_size=3)

        x = ReLU()(x)
        x = sep_bn(x, filters=728, kernel_size=3)

        x = ReLU()(x)
        x = sep_bn(x, filters=728, kernel_size=3)

        tensor = Add()([tensor, x])

    return tensor

In [6]:
def exit_flow(tensor):

    x = ReLU()(tensor)
    x = sep_bn(x, filters=728, kernel_size=3)

    x = ReLU()(x)
    x = sep_bn(x, filters=1024, kernel_size=3)

    x = MaxPool2D(pool_size=(3, 3), strides=2, padding="same")(x)

    tensor = conv_bn(tensor, filters=1024, kernel_size=1, strides=2)

    x = Add()([tensor, x])

    x = sep_bn(x, filters=1536, kernel_size=3)
    x = ReLU()(x)

    x = sep_bn(x, filters=2048, kernel_size=3)
    x = ReLU()(x)

    x = GlobalAvgPool2D()(x)

    x = Dense(units=15, activation="softmax")(x)

    return x

In [7]:
input = Input(shape=(299, 299, 3))
x = entry_flow(input)
x = middle_flow(x)
output = exit_flow(x)

xce = tf.keras.models.Model(inputs=input, outputs=output)

In [11]:
xce.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 150, 150, 32)         896       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 150, 150, 32)         128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 re_lu (ReLU)                (None, 150, 150, 32)         0         ['batch_normalization[0][0

In [12]:
def train_dg(path):

    train = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    vertical_flip=True,
    rescale=1./255,
    )

    train = train.flow_from_directory(
    directory=path,
    batch_size=32,
    target_size=(224, 224),
    class_mode="categorical"
    )
    return train

In [13]:
def val_dg(path):

    test = ImageDataGenerator(
        rescale=1./255
    )

    test = test.flow_from_directory(
    directory=path,
    batch_size=32,
    target_size=(224, 224),
    class_mode="categorical"
    )
    return test

In [14]:
train_datagen = train_dg(r"E:\Dataset\wheat disease\data\train")

Found 13104 images belonging to 15 classes.


In [15]:
test_datagen = val_dg(r"E:\Dataset\wheat disease\data\valid")

Found 300 images belonging to 15 classes.


In [17]:
xce.compile(
        loss = "categorical_crossentropy",
        optimizer="adam",
        metrics=["accuracy"]
    )

In [18]:
call = tf.keras.callbacks.EarlyStopping(
    patience=3, 
    monitor="accuracy"
)

In [20]:
xce.fit(
    x=train_datagen,
    batch_size=32,
    epochs=10,
    callbacks=call,
    validation_data=test_datagen
)

Epoch 1/10


  3/410 [..............................] - ETA: 3:13:23 - loss: 2.7623 - accuracy: 0.1562

KeyboardInterrupt: 